In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
# import re
# df = pd.read_csv('/content/drive/MyDrive/데이터/명언 홈페이지_크롤링(1~600페이지-전처리완료).txt')
# df['내용'] = df['내용'].apply(lambda x: re.sub('[^ㄱ-ㅎㅏ-ㅣ-가-힣0-9. ]', '', str(x)))
# df.to_csv('/content/drive/MyDrive/데이터/명언 홈페이지_크롤링(1~600페이지-전처리완료).txt', index = False)

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import PreTrainedTokenizerFast

def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer = tokenizer,
        mlm = mlm,
    )
    return data_collator

def train(train_file_path, model_name, output_dir, overwrite_output_dir,
          per_device_train_batch_size, num_train_epochs, save_steps):
    tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name,
                bos_token = '</s>', eos_token = '</s>', unk_token = '<unk>',
                pad_token = '<pad>', mask_token = '<mask>')
    train_dataset = load_dataset(train_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)

    tokenizer.save_pretrained(output_dir, legacy_format = False)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    model.save_pretrained(output_dir)

    training_args = TrainingArguments(
        output_dir = output_dir,
        overwrite_output_dir = overwrite_output_dir,
        per_device_eval_batch_size = per_device_train_batch_size,
        num_train_epochs = num_train_epochs,
    )

    trainer = Trainer(
        model = model,
        args = training_args,
        data_collator = data_collator,
        train_dataset = train_dataset,
    )

    trainer.train()
    trainer.save_model()

train_file_path = '/content/drive/MyDrive/데이터/명언 홈페이지_크롤링(전처리완료).txt'
model_name = 'skt/kogpt2-base-v2'
output_dir = '/content/drive/MyDrive/Colab Notebooks/팀프로젝트/빅데이터 지능형 서비스과정(최종프로젝트)/KoGPT2_FineTunning(Wise)'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 20.0
save_steps = 500

train(train_file_path = train_file_path,
      model_name = model_name,
      output_dir = output_dir,
      overwrite_output_dir = overwrite_output_dir,
      per_device_train_batch_size = per_device_train_batch_size,
      num_train_epochs = num_train_epochs,
      save_steps = save_steps
)


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/데이터/명언 홈페이지_크롤링(전처리완료).csv')
df['한글명언'] = df['명언'].apply(lambda x: x.split('\n')[0])
df.head()

In [ ]:
df['시작문'] = ['값 없음' for _ in range(len(df['한글명언']))]
for i in range(len(df['한글명언'])):
    if len(df['한글명언'][i].split(' ')) >= 3:
        df['시작문'][i] = df['한글명언'][i].split(' ')[0] + ' ' + df['한글명언'][i].split(' ')[1] + ' ' + df['한글명언'][i].split(' ')[2]
    elif len(df['한글명언'][i].split(' ')) >= 2:
        df['시작문'][i] = df['한글명언'][i].split(' ')[0] + ' ' + df['한글명언'][i].split(' ')[1]
    else:
        df['시작문'][i] = df['한글명언'][i].split(' ')[0]
df.head()

In [ ]:
start_word = list(set(list(df['시작문'])))[1:] 
df['첫문장'] = df['한글명언'].apply(lambda x: x.split('.')[0])
start_sentence = list(set(list(df['첫문장']))) 
len(start_word), len(start_sentence)

(13198, 13284)

In [ ]:
import random
n = random.randint(0, len(start_word))
start_word[n], start_sentence[n]

('사랑은 상실이며 단념이다.', '인간의 미래는 인간의 마음에 있다')

In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
from tqdm.notebook import tqdm

def load_model(model_path):
  model = GPT2LMHeadModel.from_pretrained(model_path)
  return model

def load_tokenizer(tokenizer_path):
  tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_path)
  return tokenizer

def generate_text(sequence, max_lenth):
  model_path = '/content/drive/MyDrive/Colab Notebooks/팀프로젝트/빅데이터 지능형 서비스과정(최종프로젝트)/KoGPT2_FineTunning(Wise)'
  model = load_model(model_path)
  tokenizer = load_tokenizer(model_path)
  ids = tokenizer.encode(f'{sequence},', return_tensors = 'pt')
  final_outputs = model.generate(
      ids,
      do_sample = True,
      max_length = max_length,
      pad_token_id = model.config.pad_token_id,
      tok_k = 25,
      top_p = 0.90,
      repetition_penalty = 1.5,
  )
  return tokenizer.decode(final_outputs[0])
  # return tokenizer.decode(final_outputs[0], skip_special_tokens = True)

import random
n = random.randint(0, len(start_word))
sequence = '11월 4일에 헬스장에 가서 운동을 했다.'
# sequence = df['한글명언'][n]
refer = df['인물'][n]
# sequence = start_sentence[n]
max_length = 256 
sentence_list = []
print('input : ' + sequence + ' ' + refer)
# for i in tqdm(range(5)):
sentence_list.append(generate_text(sequence, max_length))
sentence = generate_text(sequence, max_length)

In [ ]:
# sentence_extract_one_list1 = []
# sentence_extract_one_list2 = []
# for i in range(5):
#   sentence_list[i] = sentence_list[i].replace('\n', ' ')
#   sentence_extract_one_list1.append(sentence_list[i].split('.')[0] + '.' + sentence_list[i].split('.')[1] + '.')
#   sentence_extract_one_list2.append(sentence_list[i].split(', ')[1])
# sentence_extract_one_list1 # 2~3개 단어단위 입력시 가공
# sentence_extract_one_list2 # 문장단위 입력시 가공

# ch_sentence = sentence.split(sequence + ', ')[1:]
# ch_sentence_list = ch_sentence[0].split('\n')
# print(f'입력문장 : {sequence}')
# for i in range(len(ch_sentence_list)):
#   print(f'출력문장{i+1} : {ch_sentence_list[i]}')

ch_sentence = sentence.split('sequence' + ', ')
ch_sentence_list = ch_sentence[0].split('\n')[1:]
print(f'입력 명언 : {sequence} {refer}')
for i in range(len(ch_sentence_list)):
  print(f'출력 문장 : {ch_sentence_list[i]}')

입력 명언 : 11월 4일에 헬스장에 가서 운동을 했다. 커트 코베인
출력 문장 : 타인의 권리는 권력이나 부에 의하지 않고는 가질 수 없다. 그러나 타인의 권리의리는 존중되어야 한다.
출력 문장 : 인생은 투쟁과 변화와 단호한 결정으로 다져진다.
출력 문장 : 삶은 타협할 수가 없어요. 서로가 익숙해져버린 삶에 대해 고통스러워지는 거죠.
출력 문장 : 다들 행복해 질거야. 그러니까 시작만 하고 끝나는게 아니라 여러분이 만들어 놓은 일상의 모습에 놀라워 하죠. 그 모든 것에 일일이 손을 대지 않아도 돼요.
출력 문장 : 이 세상에 있는 그대로의 모습으로 당신 자신을 볼 수는 없습니다. 당신의 모습이 되어 다른 사람들의 모습을 보고 있을 수도 있고 또 그런 경험을 통해 다시 자신의 모습 변화를 시도 할수도 있습니다. 하지만 이러한 활동들은 자신만의 방식으로 미 사회를 변화시킬수는 없습니다.
출력 문장 : 중요한건 삶을 살아갈 힘을 얻는 일이란 아니다. 인간은 무엇인가를 해야 한다. 무엇을 먹느냐에 대한 우리의 의지이다.
출력 문장 : 자신을 극복하는 힘이 생긴다.
출력 문장 : 무엇을 잘 하는 건 어렵지만 그것을 즐기는 것만으로는 부족하다. 남에게 행복을 느끼게 되는 법을 알지 못하는 인생이 필요하다.
출력 문장 : 시도하지 않는 삶은 즐거운 것이다.
출력 문장 : 하루를 유익하게 보낸다는 것은 얼마나 큰 가치가 있다. 한 번 뿐이다.
출력 문장 : 평화는 두 사람 모두가 행복한 인생을 만든다.
출력 문장 : 웃음은 작은 웃음 속에 있다.
출력 문장 : 잘 견디기엔 많은 말이다. 한번밖에 없다.
출력 문장 : 사랑하는 데 1년 걸음을 짧게 느껴진다.
출력 문장 : 진 열 남을 웃지 마라.
출력 문장 : 두 마음에는 오래 가지 말라
